#### CIE4604 Simulation and Visualization
# Assignment 2 - GNSS-Reflectometry simulation

GPS and other GNSS satellite signals are reflected by the Earth’s oceans. A GNSS-R(eflectrometry) receiver on board a satellite in a low Earth orbit (LEO) is able to receive both the **direct** signal from a GNSS satellite, as well as the signal **reflected** by the surface of the Earth. This will provide very useful information on the Sea state and level in particular

![Remote Sensing geometry of GNSS-R (From: Jales, 2012](Jales2012.png)

For this assignment you are given the task to simulate a GNSS-R(eflectometry) experiment on board the International Space Station (ISS). 

All the Python scripts can be found in `CIE4604-M2-python` that you can download from Brightspace. For the assignment you need the following modules
- `tleplot.py`
- `crsutil.py`
- `specularpoint.py`

You will also need to have `simplekml` installed (See exercise 6). 

The `CIE4604-M2-python` download comes with 8 exercises in Jupyter notebooks. We suggest that you study the Jupyter notebooks first and do the exercises. These already contain a lot of useful material for the assignment, some of which can be re-purposed for the assignment. An other option is to do this assignment in parallel with the exercises.

## Description of the GNSS-R experiment on board the ISS

For the experiment a GNSS-R receiver will be installed on board the ISS. A GNSS-R receiver is able to

1.	Receive the direct signal from Global Navigation Satellite Systems (GNSS), such as the US GPS, Russian GLONASS, European GALILEO and Chinese BEIDOU satellites,
2.	Receive reflected GNSS signals from the Earth surface.

The path delay difference between direct and reflected signal is a measure for the Sea-surface height. However, by recording the full reflected wave form also other parameters about the sea state, such as wave-height and direction, can be retrieved.

The GNSS-R receiver on board the ISS uses two antenna’s, one **upward looking** standard **omnidirectional** antenna for the **direct** signals, and a specially designed **downward looking** antenna for the **reflected** signals. The opening angle of downward antenna (field of view) is not yet decided: the final value will depend on the outcome of the simulation, and will be a trade-off between received power and coverage.

## Breakdown in sub-tasks

The first step of any task is to break it down in more manageable sub-tasks. Normally this is something that you have to do yourself, but here, for this assignment, we will give you the task breakdown. The sub-tasks you have to perform are:

1. Simulate and visualize the ISS orbit trajectory
2. Simulate and visualize the GPS satellite orbits
3. Simulate and visualize GNSS signals from the upward looking omnidirectional antenna on the ISS (direct signals)
4. Simulate and visualize GNSS signals from the down looking antenna on the ISS (reflected signals)

More details on each sub-task can be found in the following cells.

## 1. Simulate and visualize the ISS orbit trajectory

Simulate and visualize the orbit trajectory of the International Space Station ISS. You may use Two-Line Elements (TLE) for ISS (Zarya) that you can find on Celestrak. The questions need to be answered in this sub-task:

a.	What are the main limitations of using GNSS-R from the ISS for the spatial and temporal coverage?
Hint: To answer this question you have to plot the ISS ground tracks (sub-satellite tracks), for which you may use `pltgroundtracks` from the `tleplot` Python module. 

b.	What is the effect of the opening angle (field of view) of the downward looking antenna on the spatial coverage? As part of your answer compute the area of coverage (on the Earth) for an opening angle (field of view) of 40 degrees and of 100 degrees. You may report the size of the area of coverage in degrees. Hint: Some background information is given in exercise 7 `CIE4604_M2_Exercise7_aoc.ipynb` notebook.

c.	Over how many hours or days do you have to carry out the simulation so that all areas, that can possibly be observed, are observed?  

In all steps we expect clear answers with a brief explanation. Include appropriate plots and visualizations wherever necessary.

In [55]:
# Enter here your code to plot the ISS ground tracks (sub-satellite tracks) using 
# pltgroundtracks from the tleplot Python module.
#
# If you wish, you can add more cells.

import tleplot as tle

In [56]:
tle.tleget('stations')
tleERS = tleplot.tleread('stations.txt')

Saving TLE set to default name
TLEGET: Downloaded stations.txt from http://celestrak.com/NORAD/elements/
Saved TLE to stations.txt

Satellite              Reference_Epoch    a [km]    ecc [-]  inc [deg] RAAN [deg] argp [deg]    E [deg]    Period

ISS (ZARYA)             2021-324.06458   6798.96  0.0004633    51.6442   291.9657   219.6337   260.2446  01:32:59
AEROCUBE 12A            2021-323.81366   6844.46  0.0007108    51.6382   120.0712     5.3354   354.7705  01:33:55
AEROCUBE 12B            2021-323.91190   6846.81  0.0007260    51.6390   122.1795     3.6599   356.4428  01:33:58
LEMUR-2-VU              2021-324.10943   6835.66  0.0006094    51.6392   108.0625    13.1245   346.9902  01:33:44
LEMUR-2-ALEXANDER       2021-323.67803   6834.81  0.0006038    51.6389   108.9571    11.0600   349.0513  01:33:43
LEMUR-2-YUASA           2021-323.82163   6835.73  0.0006255    51.6388   109.8888    25.3103   334.8191  01:33:45
LEMUR-2-TOMHENDERSON    2021-324.13755   6837.18  0.0006650    51.639

In [59]:
isat,name = tle.tlefind(tleERS,'ISS (ZA')

tle.tleplot1(tleERS,)

Found 1 satellites:
 ISS (ZARYA)  (0)


In [54]:

print(isat,name)

[0] ['ISS (ZARYA)']


*Enter here your answers using markdown. Use one cell for each subquestion*

## 2. Simulate and visualize the GPS satellite orbits

Simulate and visualize the GPS orbits. You may again use TLE from Celestrak. Deliverables for this sub-task are:

a.	Plot of the GPS ground tracks.  You may again use `pltgroundtracks` from the `tleplot` module. Please think carefully about how to plot this data in a nice and concise manner, as there are about 30 operational GPS satellites.

b.	Polar plot (skyplot), using the azimuth and zenith angles, for a station on the Earth (e.g. Delft).  Hint: You may use the function `skyplot` from the `tleplot`  module. 

c.	Plot with the number of GPS satellite in view as function of time for the same ground station. Consider carefully how you define visibility and which value you use as elevation cut-off angle. Hint: As an intermediate step, create a 2D boolean array with an axis for satellite, and an axis for time, with True whenever a satellite is visible.

You may use a period of one day for these plots.

In [ ]:
# enter here your code and run the cell to produce the result, if you wish you can use more cells

*Enter here your answers using markdown. Use one cell for each subquestion.*

## 3. Simulate and visualize direct signals

Simulate and visualize GNSS signals from the **upward looking omnidirectional** antenna on the ISS (direct signals). The opening angle of the upward looking antenna is 160 degrees (comparable to a 10 degree elevation cut-off angle). For this sub-task you have to use the simulated ISS and GPS positions from the previous sub-tasks. Questions to answer are:

a.	How many GPS satellites are on average in view from the upward looking antenna? Illustrate this by a plot with the number of GPS satellite in view as function of time from the ISS (as in 2c). How does this compare to a station on the Earth (e.g. Delft)?

b.	What is the average duration of GPS satellite visibility, in minutes from start to end for each pass, from the ISS? Include a plot that clearly illustrates this answer. Again, how does this compare to the a station on the Earth?

c.	Visualize the ISS, GPS and observations from the upward looking omnidirectional antenna in Google Earth using the `simplekml` Python module. You may get some inspiration from exercise 6 `CIE4604-M2-Example6_kml.ipynb` notebook. Make separate kml folders (within the same kmz file) for ISS, GPS and direct observations. For the report (Jupyter notebook), include a screenshot showing trajectories for one day, but the direct observations only for a single epoch.

You may use a period of one day for the plots. 

In [ ]:
# enter here your code and run the cell to produce the result, if you wish you can use more cells

*Enter here your answer using markdown. Use one cell for each sub-question. For question 3c do not forget to include the screenshot, you can do this with syntax like `![](screenshot.png)`*.

## 4. Simulate and visualize reflected signals


Simulate and visualize GNSS signals from the **downward looking** antenna on the ISS (reflected signals). The opening angle of the downward looking antenna is 40 degrees or 100 degrees.

a.	Make a function to compute the position of  the specular reflection points that are within the field of view (opening angle) of the down looking GPS antenna on board the ISS at a specified moment in time. 

Input for this function is:  the position of the ISS and  GPS satellite positions at a specific time (one epoch only), and the opening angle of the downward looking antenna (40 or 100 degrees). The output should be the position of the specular reflection points (zero, one or more) that fall within the opening angle. For the actual computation you may call the function `specularpoint` from the `specularpoint.py` module that is part of the `CIE4604_M2_Python.zip` distribution. This is also explained in more detail in exercise 8 `CIE4604_M2_Example8_sp.ipynb` notebook.

b.	Make a plot with the specular reflection points, ISS and GPS ground tracks in Python for a period of several minutes for two different values of the opening angle.

c.	Visualize the specular reflection points and reflected signals in Google Earth for two different values of the opening angle (40 degrees and 100 degrees), and add this to the kmz file of question 3c. Make a new kml folder (within the same kmz file) for each opening angle. 

Include in your report a screenshot of Google Earth showing the ISS, GPS and specular reflection point trajectories for a period of several minutes, while showing the direct and reflected signals only for a single epoch.

In [ ]:
# enter here your code and run the cell to produce the result, if you wish you can use more cells

*Enter here your answer using markdown. Use one cell for each sub-question. For question 3c do not forget to include the screenshot, you can do this with syntax like `![](screenshot.png)`*.

# Deliverables

The deliverables for this assignment are 

1.	Pdf of your Jupyter notebook or short report (pdf)  with a description of the GNSS-R simulator, plots of the results and answers to the questions.
2.	KMZ file (for display in Google Earth), one kmz file only, for question 3c and 4c combined.
3.	Jupyter notebook or zip file with the code.

The above deliverables have to be submitted electronically  through Brightspace.


[End of this Jupyter notebook]